In [112]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [113]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [114]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [115]:
# Cell 3

wallet = KeyWallet.load("../../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../../keystores/balanced_test.json", key_data)
with open("../../keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("../../keystores/staking_test.json", key_data)


In [116]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
780838304.445043


In [117]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
4171640.2254082


In [118]:
user1 = KeyWallet.load("../../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

189999.8499788
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
208479.757389
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [119]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [120]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        deploy.remove('bnXLM')
        deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "",  # ICX Station
            "",  # iBriz-ICONOsphere
            "",  # Mousebelt
            "",  # Parrot 9
            ""}  # ICONDAO
    else:
        return
    
    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    print(result)
    return result

In [121]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

../../core_contracts/governance.zip
Status: 1

../../core_contracts/loans.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/dividends.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../core_contracts/reserve.zip
Status: 1

../../core_contracts/daofund.zip
Status: 1

../../core_contracts/rewards.zip


...

Retrying in 1 seconds...
Status: 1

../../core_contracts/staking.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

../../token_contracts/sicx.zip


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
Status: 1

------------------------------------------------------------------------------------------------------------------
{'loans': {'zip': '../../core_contracts/loans.zip', 'SCORE': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e'}, 'staking': {'zip': '../../core_contracts/staking.zip', 'SCORE': 'cx8261e32f663858db1a5ff54cb5bb255cb9c9a663'}, 'dividends': {'zip': '../../core_contracts/dividends.zip', 'SCORE': 'cxfd445b447d25816deb85f0f6c68723649fc68d1b'}, 'reserve': {'zip': '../../core_contracts/reserve.zip', 'SCORE': 'cx4278fbf4b4317a1214d68a75a4a99b828bd43218'}, 'daofund': {'zip': '../../core_contracts/daofund.zip', 'SCORE': 'cx40ec8990535e25cf00d11020a0e126e3c1cb875d'}, 'rewards': {'zip': '../../core_contracts/rewards.zip', 'SCORE': 'cxda49d813b7532d062dac5fa6e6393de98903def0'}, 'dex': {'zip': '../../core_contracts/dex.zip', 'SCORE': 'cx9c78b71ab3d2422e53f935e3e149ee9089dc5030'}, 'governance': {'zip': '../../core_contracts/governance.zip', 'SCORE': 'cx716db4

In [122]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', 'staking': 'cx8261e32f663858db1a5ff54cb5bb255cb9c9a663', 'dividends': 'cxfd445b447d25816deb85f0f6c68723649fc68d1b', 'reserve': 'cx4278fbf4b4317a1214d68a75a4a99b828bd43218', 'daofund': 'cx40ec8990535e25cf00d11020a0e126e3c1cb875d', 'rewards': 'cxda49d813b7532d062dac5fa6e6393de98903def0', 'dex': 'cx9c78b71ab3d2422e53f935e3e149ee9089dc5030', 'oracle': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7', 'sicx': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0', 'bnUSD': 'cxef08836eb84232f62493cf1fd5312729d77bf9c5', 'baln': 'cx16d399ad903e9d872ebf488f35629a610a029de5', 'bwt': 'cx7fe1a8f194590e420aee85bc4d2c0d1f7ab725d5'}} on the governance contract.
------------------------------------------------------------------------------------------------------------------
********************

In [123]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', 'indexed': ['ContractActive(str,str)', 'Loans'], 'data': ['Active']} 

------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
------------------------------------------------------------------------------------------------------------------
Calling delegate, with parameters {'_delegations': [{'_address': 'hx000e0415037ae871184b2c7154e5924ef2bc075e', '_votes_in_per': '50000000000000000000'}, {'_address': 'hx9eec61296a7010c867ce24c20e69588e2832bc52', '_votes_in_per': '50000000000000000000'}]} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'governance|launchBalanced|{}': {'txHash': '0x975c9d704fea96efd30ad94feeee7fcac73c624d25e761dae96fb7101ba9d046', 'blockHeight': 266143, 'blockHash': '0xa83d46f72ea4f8edca5b854c3de02a5f0503f7fbb9ffdad7b03fcd90d44a7230', 'txIndex': 1, 'to'

In [88]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'staking'
update = 1
params = {}
if update == 0:
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    deploy_SCORE(contract, params, btest_wallet, update)

Updating staking with params: {} to custom. Proceed (Yes/No)? Yes
../../core_contracts/staking.zip
Status: 1



In [48]:
send_tx('staking', 0, 'toggleStakingOn', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling toggleStakingOn, with parameters {} on the staking contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************


{'txHash': '0xc40f28140c7b131e95262cfdbd05829d1f935dd1fca14fb9d75e022bd3988f40',
 'blockHeight': 255523,
 'blockHash': '0xd3a1d00502b1aea97adb74589c2f9134b667a244d3b2b58d06757841877210aa',
 'txIndex': 1,
 'to': 'cxdb102ae83b69bc45f4dd195f5d48453d44d0710e',
 'stepUsed': 130680,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 130680,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [49]:
send_tx('governance', 0, 'balanceToggleStakingEnabled', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling balanceToggleStakingEnabled, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************


{'txHash': '0xe1e2f1c818fff8da04935e073d4ad35fd590e279cabc3001e491c435ae0c4a99',
 'blockHeight': 255524,
 'blockHash': '0x2bd4039b2db25fc9e6e2b543170177ba1538ac26037bbd4ebfbcea1f7e47eaef',
 'txIndex': 1,
 'to': 'cx0081a3ee4e80db52bd53ffe4849ed3cdd383cfd3',
 'stepUsed': 158080,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 158080,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [124]:
# 1.no bad debt for asset, retired from account user2 without a position on Balanced.
call_tx('loans', 'getAccountPositions', {'_owner': user1.get_address()})
call_tx('loans', 'getAccountPositions', {'_owner': user2.get_address()})


------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}
------------------------------------------------------------------------------------------------------------------
Reading getAccountPositions, with parameters {'_owner': 'hx7a1824129a8fe803e45a3aae1c0e060399546187'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
{'message': 'That address has no outstanding loans or deposited collateral.'}


{'message': 'That address has no outstanding loans or deposited collateral.'}

In [125]:
call_tx('loans', 'getMaxRetireAmount', {'_symbol': 'bnUSD'})


------------------------------------------------------------------------------------------------------------------
Reading getMaxRetireAmount, with parameters {'_symbol': 'bnUSD'} on the loans contract.
------------------------------------------------------------------------------------------------------------------
0x0


'0x0'

In [126]:
send_tx('loans', 2000*ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 500 * ICX}, user1)


------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 500000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', 'cx8261e32f663858db1a5ff54cb5bb255cb9c9a663', '0x6c6b935b8bbd400000'], 'data': []} 

{'scoreAddress': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0', 'indexed': ['Mint(Address,int,bytes)', 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e'], 'data': ['0x6c6b935b8bbd400000', '0x4e6f6e65']} 

{'scoreAddress': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', '0x6c6b935b8bbd400000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx8261e32f663858db1a5ff54cb5bb255cb9c9a663', 'indexed': ['TokenTransfer(Address,int,str)', 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', '0x6c6b935

{'txHash': '0x7782932bf086928f2c659daac4c71d01ae796fd8bfc0032222734e62ed19bf57',
 'blockHeight': 266159,
 'blockHash': '0x3f03dc9751c5c00c6e1067f812fd1761ff85c7fcf1eccb355ab1dde232c4dbfa',
 'txIndex': 1,
 'to': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e',
 'stepUsed': 880020,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 880020,
 'eventLogs': [{'scoreAddress': 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e',
    'cx8261e32f663858db1a5ff54cb5bb255cb9c9a663',
    '0x6c6b935b8bbd400000'],
   'data': []},
  {'scoreAddress': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0',
   'indexed': ['Mint(Address,int,bytes)',
    'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e'],
   'data': ['0x6c6b935b8bbd400000', '0x4e6f6e65']},
  {'scoreAddress': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000000000000000000000000',
    'cxff18

In [23]:
send_tx('loans', 0, 'returnAsset', {'_symbol': 'bnUSD', '_value': 50 * ICX}, wallet)


------------------------------------------------------------------------------------------------------------------
Calling returnAsset, with parameters {'_symbol': 'bnUSD', '_value': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['Burn(Address,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2b5e3af16b1880000']} 

{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hx0000000000000000000000000000000000000000', '0x2b5e3af16b1880000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxfeaee564d1cf035aa61184049badb51639fe7107', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'bnUSD', '0x2b5e3af16b1880000'], 'data': ['Loan of 50000000000000000000 bnUSD repaid to Balanced.']} 

{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx2780eeb8c800ac9886786baae281c3d23

{'txHash': '0x052d7d0ca14322080d9046481b0fce424bf4e69b42af76044e8e791be0645c14',
 'blockHeight': 172185,
 'blockHash': '0x71580799e364f02a4d24bdda697bd3663dcf7ba6ff163621ab07a6bcb3ad978b',
 'txIndex': 1,
 'to': 'cxfeaee564d1cf035aa61184049badb51639fe7107',
 'stepUsed': 358340,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 358340,
 'eventLogs': [{'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9',
   'indexed': ['Burn(Address,int)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'],
   'data': ['0x2b5e3af16b1880000']},
  {'scoreAddress': 'cx084e59320817d17dc6e05e4f9a7fe435716aeba9',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb',
    'hx0000000000000000000000000000000000000000',
    '0x2b5e3af16b1880000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxfeaee564d1cf035aa61184049badb51639fe7107',
   'indexed': ['LoanRepaid(Address,str,int,str)',
    'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb',
    'bnUSD',
    '0x2b

In [128]:
test_cases = {
        "stories": [
            {
                "description": "User2 tries to retire 10 bnusd",
                  "actions": {
                      "sender":"user2",
                      "first_meth": "transfer",
                      "second_meth":"returnAsset",
                      "deposited_icx":"0",
                      "first_params":{'_to': user2.get_address(), '_value': 20 * ICX},
                      "second_params":{"_symbol": "bnUSD", '_value': 5 * ICX},
                      "expected_status_result":"1"
              }
            }
      ]
    }

for case in test_cases['stories']:
    print(case['description'])
    _to = contracts['bnUSD']['SCORE']
    meth1 = case['actions']['first_meth']
    meth2 = case['actions']['second_meth']
    val = int(case['actions']['deposited_icx'])
    data1 = case['actions']['first_params']
    first_params = {"_to": data1['_to'], "_value": data1['_value']}

    data2 = case['actions']['second_params']
    second_params = {'_symbol': data2['_symbol'], '_value': data2['_value']}

    send_tx('bnUSD', 0 ,meth1 , first_params, user1)
    sleep(2)
    res = send_tx('loans', 0 ,meth2 , second_params, user2)
    assert res['status'] == int(case['actions']['expected_status_result']), 'Retired amount is greater than the current maximum allowed'
    print('Test case passed')

User2 tries to retire 10 bnusd
------------------------------------------------------------------------------------------------------------------
Calling transfer, with parameters {'_to': 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '_value': 20000000000000000000} on the bnUSD contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxef08836eb84232f62493cf1fd5312729d77bf9c5', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '0x1158e460913d00000'], 'data': ['0x4e6f6e65']} 

------------------------------------------------------------------------------------------------------------------
Calling returnAsset, with parameters {'_symbol': 'bnUSD', '_value': 5000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxef08836eb84232f62493cf1fd5312729d77bf9c5', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxef08836eb84232f62493cf1fd5312729d77bf9c5', 'indexed': ['Burn(Address,int)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187'], 'data': ['0x4563918244f40000']} 

{'scoreAddress': 'cxef08836eb84232f62493cf1fd5312729d77bf9c5', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', 'hx0000000000000000000000000000000000000000', '0x4563918244f40000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxabadeed2005b2200eaf3c7cf5e3706e5ed4940e0', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxff18dccc14c5a05dba4c7b3c2d183898b3ebbf0e', 'hx7a1824129a8fe803e45a3aae1c0e060399546187', '0x2948b7e2